## Load Dataset

In [73]:
import torch

In [76]:
# with open("instructions.txt", "r", encoding="utf-8") as f:
#     text = f.read()
import pandas as pd
train = pd.read_parquet('../../data/train/train-00000-of-00001.parquet',columns=['question','query'])

text = train.sample(frac=0.2).reset_index(drop=True)

# train_sample.drop('source', axis=1,inplace=True)

text

,question,query
0,Of all players with an overall rating greater ...,"SELECT preferred_foot , count(*) FROM Player_..."
1,Count the number of captains that have each rank.,"SELECT count(*) , rank FROM captain GROUP BY ..."
2,Show ids of students who don't play video game.,SELECT StuID FROM Student EXCEPT SELECT StuID ...
3,Which papers' first author is affiliated with ...,SELECT t3.title FROM authors AS t1 JOIN author...
4,"What are the first names, office locations, an...","SELECT T2.emp_fname , T4.prof_office , T3.cr..."
...,...,...
1395,What is the year and semester with the most co...,"SELECT semester , YEAR FROM SECTION GROUP BY ..."
1396,"What is the product, chromosome and porphyria ...","SELECT product , chromosome , porphyria FROM..."
1397,Find id of candidates whose assessment code is...,SELECT candidate_id FROM candidate_assessments...
1398,How many friends does Dan have?,SELECT count(T2.friend) FROM Person AS T1 JOIN...


In [77]:
type(text), text[:3], len(text)

(pandas.core.frame.DataFrame,
                                             question  \
 0  Of all players with an overall rating greater ...   
 1  Count the number of captains that have each rank.   
 2    Show ids of students who don't play video game.   
 
                                                query  
 0  SELECT preferred_foot ,  count(*) FROM Player_...  
 1  SELECT count(*) ,  rank FROM captain GROUP BY ...  
 2  SELECT StuID FROM Student EXCEPT SELECT StuID ...  ,
 1400)

In [78]:
# import random

# seed = 42

# random.seed(seed)
# random.shuffle(text)  # this could also be a parameter

In [79]:
train_dataset = text[:-100]
eval_dataset = text[-100:]

In [80]:
train_dataset, eval_dataset

(                                               question  \
 0     Of all players with an overall rating greater ...   
 1     Count the number of captains that have each rank.   
 2       Show ids of students who don't play video game.   
 3     Which papers' first author is affiliated with ...   
 4     What are the first names, office locations, an...   
 ...                                                 ...   
 1295  Give the names of people who did not participa...   
 1296  List the hardware model name for the phones th...   
 1297  Find the last name of the latest contact indiv...   
 1298  Which student visited restaurant most often? L...   
 1299                         How many events are there?   
 
                                                   query  
 0     SELECT preferred_foot ,  count(*) FROM Player_...  
 1     SELECT count(*) ,  rank FROM captain GROUP BY ...  
 2     SELECT StuID FROM Student EXCEPT SELECT StuID ...  
 3     SELECT t3.title FROM authors AS t1 

In [81]:
def prompt_no_input(row):
    return ("Below is an instruction that describes a task. "
            "Write a response that appropriately completes the request.\n\n"
            "### Instruction:\n{question}\n\n### Response:\n").format_map(row)

In [82]:
row = text.iloc[0,:]
print(prompt_no_input(row))

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Of all players with an overall rating greater than 80, how many are right-footed and left-footed?

### Response:



In [83]:
def prompt_input(row):
    return ("Below is an instruction that describes a task, paired with an input that provides further context. "
            "Write a response that appropriately completes the request.\n\n"
            "### Instruction:\n{question}\n\n### Input:\n{input}\n\n### Response:\n").format_map(row)

In [84]:
print(prompt_no_input(row))

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Of all players with an overall rating greater than 80, how many are right-footed and left-footed?

### Response:



In [85]:
def create_alpaca_prompt(row):
    return prompt_no_input(row) if row["input"] == "" else prompt_input(row)

In [86]:
[row for i, row in train_dataset.iterrows()]

[question    Of all players with an overall rating greater ...
 query       SELECT preferred_foot ,  count(*) FROM Player_...
 Name: 0, dtype: object,
 question    Count the number of captains that have each rank.
 query       SELECT count(*) ,  rank FROM captain GROUP BY ...
 Name: 1, dtype: object,
 question      Show ids of students who don't play video game.
 query       SELECT StuID FROM Student EXCEPT SELECT StuID ...
 Name: 2, dtype: object,
 question    Which papers' first author is affiliated with ...
 query       SELECT t3.title FROM authors AS t1 JOIN author...
 Name: 3, dtype: object,
 question    What are the first names, office locations, an...
 query       SELECT T2.emp_fname ,  T4.prof_office ,  T3.cr...
 Name: 4, dtype: object,
 question    Where is the club "Hopkins Student Enterprises...
 query       SELECT clublocation FROM club WHERE clubname  ...
 Name: 5, dtype: object,
 question    Show the nominees that have nominated musicals...
 query       SELECT Nominee FRO

In [87]:
train_prompts = [prompt_no_input(row) for i, row in train_dataset.iterrows()]
eval_prompts = [prompt_no_input(row) for i, row in eval_dataset.iterrows()]

In [88]:
print(train_prompts[0])

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Of all players with an overall rating greater than 80, how many are right-footed and left-footed?

### Response:



In [89]:
def pad_eos(ds):
    EOS_TOKEN = "</s>"
    return [f"{row['query']}{EOS_TOKEN}" for _,row in ds.iterrows()]

In [90]:
train_outputs = pad_eos(train_dataset)
eval_outputs = pad_eos(eval_dataset)
train_outputs[0]

'SELECT preferred_foot ,  count(*) FROM Player_Attributes WHERE overall_rating  >  80 GROUP BY preferred_foot</s>'

In [91]:
train_dataset = [{"prompt":s, "output":t, "example": s + t} for s, t in zip(train_prompts, train_outputs)]
eval_dataset = [{"prompt":s, "output":t, "example": s + t} for s, t in zip(eval_prompts, eval_outputs)]

In [92]:
train_dataset[0]

{'prompt': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nOf all players with an overall rating greater than 80, how many are right-footed and left-footed?\n\n### Response:\n',
 'output': 'SELECT preferred_foot ,  count(*) FROM Player_Attributes WHERE overall_rating  >  80 GROUP BY preferred_foot</s>',
 'example': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nOf all players with an overall rating greater than 80, how many are right-footed and left-footed?\n\n### Response:\nSELECT preferred_foot ,  count(*) FROM Player_Attributes WHERE overall_rating  >  80 GROUP BY preferred_foot</s>'}

In [100]:
import tiktoken
tokenizer = tiktoken.get_encoding('gpt2')

In [101]:
tokenizer.encode("experiments are going strong!")

[23100, 6800, 389, 1016, 1913, 0]

TypeError: encode() got an unexpected keyword argument 'padding'

In [102]:
# max_sequence_len = 1024

# def pack(dataset, max_seq_len=max_sequence_len):
#     tkds_ids = tokenizer([s["example"] for s in dataset])["input_ids"]
    
#     all_token_ids = []
#     for tokenized_input in tkds_ids:
#         all_token_ids.extend(tokenized_input)# + [tokenizer.eos_token_id])
    
#     print(f"Total number of tokens: {len(all_token_ids)}")
#     packed_ds = []
#     for i in range(0, len(all_token_ids), max_seq_len+1):
#         input_ids = all_token_ids[i : i + max_seq_len+1]
#         if len(input_ids) == (max_seq_len+1):
#             packed_ds.append({"input_ids": input_ids[:-1], "labels": input_ids[1:]})  # this shift is not needed if using the model.loss
#     return packed_ds

In [110]:
from torch.utils.data import DataLoader
from transformers import default_data_collator

torch.manual_seed(123)
batch_size = 16  # I have an A100 GPU with 40GB of RAM 😎

train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    collate_fn=default_data_collator, # we don't need any special collator 😎
)

eval_dataloader = DataLoader(
    eval_dataset,
    batch_size=batch_size,
    collate_fn=default_data_collator,
    shuffle=False,
)

/Users/jagpreetsingh/ML_Projects/text-sql/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [111]:
b = next(iter(train_dataloader))
b

{}

In [114]:
from types import SimpleNamespace

gradient_accumulation_steps = 2

config = SimpleNamespace(
    model_id='my_model.pt',
    dataset_name="alpaca-gpt4",
    precision="bf16",  # faster and better than fp16, requires new GPUs
    n_freeze=24,  # How many layers we don't train, LLama 7B has 32.
    lr=2e-4,
    n_eval_samples=10, # How many samples to generate on validation
    max_seq_len=max_sequence_len, # Lenght of the sequences to pack
    epochs=3,  # we do 3 pasess over the dataset.
    gradient_accumulation_steps=gradient_accumulation_steps,  # evey how many iterations we update the gradients, simulates larger batch sizes
    batch_size=batch_size,  # what my GPU can handle, depends on how many layers are we training  
    log_model=False,  # upload the model to W&B?
    gradient_checkpointing = True,  # saves even more memory
    freeze_embed = True,  # why train this? let's keep them frozen ❄️
    seed=seed,
)

config.total_train_steps = config.epochs * len(train_dataloader) // config.gradient_accumulation_steps

{}